In [2]:
import pickle
import pandas as pd
import os
import setup

import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from tqdm.notebook import tqdm, trange
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb
from torch.utils.data.sampler import SubsetRandomSampler
import pprint
import matplotlib.pyplot as plt
import torch

from sklearn.preprocessing import MinMaxScaler
import math
from torch.profiler import profile, record_function, ProfilerActivity

from operator import itemgetter
import operator
from random import randint
# from rnn_classes import Dog, DogInput, Race, Races, GRUNet, smallGRUNet, smalll_lin_GRUNet, smalll_prelin_GRUNet
import rnn_tools.rnn_classes as rnn_classes
from rnn_tools.raceDB import build_dataset
import importlib
import datetime
from rnn_tools.model_saver import model_saver, model_saver_wandb
# import rnn_tools.training_testing_gru as training_testing_gru
from torch.nn.utils.rnn import pack_padded_sequence, pack_sequence, pad_packed_sequence,pad_sequence, unpack_sequence, unpad_sequence
import rnn_tools.training_testing_gru_double as training_testing_gru_double
from goto_conversion import goto_conversion

import rnn_tools.training_testing_gru_extra_data as training_testing_gru_extra_data
import rnn_tools.training_testing_gru_extra_data_embedding as training_testing_gru_extra_data_embedding

c:\Users\Nick\Documents\GitHub\grvmodel\python


In [3]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc.
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [ ]:
torch.set_default_tensor_type(torch.FloatTensor)

c:\Users\Nick\.conda\envs\python311\Lib\site-packages\torch\__init__.py:614: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at C:\cb\pytorch_1000000000000\work\torch\csrc\tensor\python_tensor.cpp:453.)
  _C._set_default_tensor_type(t)


In [ ]:
torch.__version__

'2.1.1'

In [20]:
os.getcwd()
import rnn_tools.raceDB
importlib.reload(rnn_tools.raceDB)
# os.chdir(r"C:\Users\Nick\Documents\GitHub\grvmodel\Python\DATA")                                                                                                                                                                                                                                                                                                                
date = datetime.datetime.strptime("2022-01-01", "%Y-%m-%d").date()
hidden_size = 32
states = ["NZ","SA","WA"]
# states = ["VIC"]
# states = ["NSW"]
states = ["NZ"]
data_file = './data/gru_inputs_kitchen_sink_new_not_simple.fth'
# data_file = './data/gru_inputs_kitchen_sink_april.fth'
data_file = './data/topaz_data_w_bsp_new.fth'
# data_file = './data/gru_inputs_kitchen_sink_topaz.fth'
raceDB = rnn_tools.raceDB.build_dataset_topaz(data_file, hidden_size,state_filter=states,date_filter=date, margin_type='sftmin',v6=True,show_stats=False)
raceDB.create_new_weights_v2()

c:\Users\Nick\Documents\GitHub\grvmodel\python
(2226749, 145)
54
Latest date = 2024-04-28 00:00:00
size after state filter (389062, 148)
(389062, 148)
Latest date = 2024-04-28 00:00:00
stats = [ 1.21535182e-01  1.00000000e+00  8.29999983e-01  0.00000000e+00
  0.00000000e+00  1.37799263e-01  3.18000000e+02  6.00000000e+00
  4.27310824e-01  2.00000000e+00  6.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  4.27310824e-01 -2.00000000e+00
  1.65551701e+01  0.00000000e+00  4.26802307e+02  4.54034662e+00
  4.61202919e-01  4.01505947e+00  7.22534227e+00  0.00000000e+00
  2.90632188e-01  3.85105252e+00  4.61202919e-01 -4.01505947e+00
  1.83189640e+01  1.60715386e-01  3.11057129e+02  4.64523792e+00
  4.44044054e-01  4.04761887e+00  6.78988075e+00  0.00000000e+00
  1.55908540e-01  1.98157144e+00  4.44044054e-01 -4.04761887e+00
  1.66556683e+01  1.97619051e-01  3.85032318e+02  4.55107260e+00
  4.56100464e-01  4.41114950e+00  7.30734158e+00  0.00000000e+00
  4.63033617e-01  7.0393385

100%|██████████| 25/25 [00:00<?, ?it/s]


race_ids=['745289958', '745289969', '745289968', '745289970', '745289967', '745289964', '745289966', '745289965', '745289963', '745289960', '745289962', '745289961', '745289959', '745616440', '745616451', '745616450', '745616449', '745616446', '745616448', '745616447', '745616442', '745616445', '745616444', '745616443', '745616441', '745290054', '745290052', '745290053', '745290051', '745289979', '745289974', '745289980', '745289982', '745290056', '745289972', '745290055', '745289971', '745290050', '745289981', '745289973', '745290061', '745290060', '745290059', '745290058', '745290057', '745289978', '745289975', '745289977', '745289976', '745589475', '745616525', '745589482', '745589483', '745589476', '745589477', '745616526', '745616533', '745616532', '745589478', '745589479', '745589481', '745589480', '745616527', '745616530', '745616531', '745616528', '745616523', '745616522', '745616529', '745616524', '749000581', '749000580', '749000582', '749000577', '749000576', '749000573', '7

In [15]:
def closure(optimizer, criterion, outs, classes):
    optimizer.zero_grad()
    loss = nn.functional.mse_loss(outs, classes)
    loss.backward()
    return loss

def model_pipeline(my_dataset=raceDB,config=None,prev_model=None, sweep=True, model_state_dict=None, prev_model_file=None, prev_model_version='450'):
    if my_dataset:
      dataset = my_dataset    
    else:
      dataset = raceDB
    # tell wandb to get started
    with wandb.init(project="NEW GRU V7 Reporting", config=config,save_code=False):
      #  access all HPs through wandb.config, so logging matches execution!
      wandb.define_metric("loss_val", summary="min")
      wandb.define_metric("accuracy", summary="max")
      wandb.define_metric("ROI < 30", summary="max")
      wandb.define_metric("val_ROI < 30", summary="max")
      wandb.define_metric("relu roi", summary="max")
      
      config = wandb.config
      pprint.pprint(config)
      pprint.pprint(config.epochs)
      print(config)
      wandb.run.log_code('rnn_tools/')
      # input_size = raceDB.get_race_input([0,1])[0].full_input.shape[0] #create fix so messy
      print(config.input_type)

      for i in config.stat_list_dict.values():
        print(i)

      reg_stat_mask = []
      for stat,flag in config.stat_list_dict.items():
          stat_flag = flag
          reg_stat_mask = reg_stat_mask+[stat_flag]

      print(reg_stat_mask)
      stat_mask = [1]*6+reg_stat_mask+[0]*80
      # stat_mask = [1]*30
      stat_mask = [1]*17+37*[0]
      data_mask = [0]*26+[1]*20+[1]*20+[1]*20+[1]*20 # Reg, Dist, Box, T_box, T_dist
      data_mask = [1]*6+reg_stat_mask+[0]*20+[0]*20+[0]*20+[0]*20 # Reg, Dist, Box,T_box, T_dist
      data_mask =  [0]*17+37*[1]
      # data_mask =  [1]*30
      data_mask_size = sum(data_mask)
      print(f"{data_mask_size=}")
      stat_mask = torch.tensor(stat_mask).type(torch.bool).to(device)
      data_mask = torch.tensor(data_mask).type(torch.bool).to(device)
      # stat_mask = torch.ones_like(stat_mask).type(torch.bool).to(device)
      input_size = sum(stat_mask)

      print(stat_mask)
      print(stat_mask.shape)


      if 'batch_days' in config.keys() and not raceDB.batches_setup:
        pass
        raceDB.create_test_split_date(config['training_date_end'],val_date='2023-07-01')
        raceDB.create_dogs_test_split_date()
        raceDB.attach_races_to_dog_inputv2() 
        raceDB.reset_hidden()
        raceDB.create_batches(batch_days=config['batch_days'], end_date = config['training_date_end'], stat_mask=stat_mask,data_mask=data_mask,gen_packed_seq=True)
        raceDB.batches_setup = True
      if config['input_type'] == 'basic':
          print('here')
          raceDB.batches['packed_x'] = raceDB.batches['packed_x_basic']
          raceDB.batches['packed_y'] = raceDB.batches['packed_y_basic']
          raceDB.batches['packed_v'] = raceDB.batches['packed_v_basic']
          # input_size = raceDB.batches['packed_x'][0].data[0].shape[0]



      print(f"{input_size=}")
      config['stat_mask_tensor'] = torch.tensor(stat_mask, dtype=torch.uint8).to(device)     
      raceDB.reset_hidden(num_layers=config['num_layers'], hidden_size=config['hidden_size'])
      model = rnn_classes.GRUNetv3_extra_embedding(input_size,config['hidden_size'], num_layers=config['num_layers'],fc0_size=config['f0_layer_size'], fc1_size=config['f1_layer_size'],data_mask_size=data_mask_size)
      optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'])
      
      config['model_name'] = model.__class__.__name__
      if model_state_dict:
        model.load_state_dict(model_state_dict)
      if prev_model_file!=None:
        print(f"Loading model {prev_model_file}, version {prev_model_version}")
        model_name = prev_model_file
        model_loc = f"models/savedmodel/{model_name}/{model_name}_{prev_model_version}.pt"
        model_data = torch.load(model_loc,map_location=torch.device('cuda:0'))
        print(model_data['model_state_dict'].keys())
        model.load_state_dict(model_data['model_state_dict'], strict=False)
        config['parent model'] = prev_model_file
        raceDB.fill_hidden_states_dict_v2(model_data['db_train'])
        model = model.to(device)
        # raceDB.load_hidden_in_dict(hidden_in_dict=model_data['hidden_ins'],output_dict=model_data['output'])
        
        optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'])
        # optimizer.load_state_dict(model_data['optim'])
      else:
        optimizer = optim.RAdam(model.parameters(), lr=config['learning_rate'])

      raceDB.to_cuda()
      criterion = nn.CrossEntropyLoss(reduction='none')
      wandb.run.name = f"{' '.join(states)}-{wandb.run.name}"
      model = model.to(device)
      print(model)

      profit_model = rnn_classes.GRUNetv3_profit(raceDB).to('cuda:0')
      raceDB.profit_model = profit_model

      raceDB.dogsDict['nullDog'].input.hidden_out = (-torch.ones(config['hidden_size']+config['f1_layer_size'])).to('cuda:0')
      training_testing_gru_extra_data_embedding.test_model_v3(model,raceDB,criterion,config=config)
      training_testing_gru_extra_data_embedding.validate_model_v3(model,raceDB,criterion,config=config)

      # try:
      #   return training_testing_gru_extra_data_embedding.train_double_v3(model, dataset, criterion, optimizer, 'na', config)
      # except (KeyboardInterrupt) as error:
      #   print(error)
      #   print("finished Early")
        
      # raceDB.create_hidden_states_dict_v2()
      # model_saver_wandb(model, optimizer, 450, 0.1, raceDB.hidden_states_dict_gru_v6,raceDB.train_hidden_dict , model_name="long nsw new  22000 RUN")


    return (model,dataset, optimizer)

In [21]:
raceDB.batches_setup = False

In [17]:
stat_list_dict = {'_dist_last__1':1,
    '_box_last__1':1,
    '_speed_avg_1':1,
    '_split_speed_avg_1':0,
    '_split_margin_avg_1':1,
    '_margin_avg_1':1,
    '_margin_time_avg_1':1,
    '_RunHomeTime_1':1,
    '_run_home_speed_1':1,
    '_first_out_avg_1':0,
    '_pos_out_avg_1':0,
    '_post_change_avg_1':0,
    '_races_1':1,
    '_wins_1':1,
    '_wins_last_1':1,
    '_weight_':1,
    '_min_time_':1,
    '_min_split_time_':1,
    '_last_start_price':0,
    '_last_start_prob':0,
}

wandb_config_static = {
    'hidden_size': 256,
    'stats': raceDB.stats_cols,
    'races': states,
    'datafile': data_file,
    'latest_date': raceDB.latest_date,
    'input_type': 'basic',
    'num_layers': 2,
    'batch_size': 750,
    'dropout': 0.3,
    'epochs': 2500,
    'learning_rate': 0.0001,
    'optimizer': 'adamW',
    'f0_layer_size': 128,
    'f1_layer_size': 64,
    'training_date_end': '2023-01-01',
    'notes':'GRU, with basic add on data looped in',
    'batch_days': 390,
    'stat_list_dict': stat_list_dict,
}

In [18]:
raceDB.reset_hidden(num_layers=wandb_config_static['num_layers'], hidden_size=wandb_config_static['hidden_size'])
torch.cuda.empty_cache()
raceDB.race_prices_to_prob()
print(raceDB.latest_date)
for race in raceDB.racesDict.values():
    race.loss = torch.tensor(1.0,requires_grad=True,device='cuda:0')  

for race in raceDB.racesDict.values():
    if race.classes.isnan().sum():
        print(race.classes.sum())
        print(race.raceid)
        race.classes = race.classes.nan_to_num(0,0,0)

2024-04-28 00:00:00


In [22]:
importlib.reload(rnn_tools.rnn_classes)
importlib.reload(rnn_classes)
importlib.reload(training_testing_gru_extra_data)
importlib.reload(training_testing_gru_extra_data_embedding)
# importlib.reload(training_testing_gru_extra_data_state_batch)
# importlib.reload(training_testing_gru_extra_data_toasty)
# importlib.reload(training_testing_gru_extra_data_toasty_og)
# (model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False)
model_pipeline(raceDB,config=wandb_config_static,sweep=False, prev_model_file='NZ-ancient-nexu-541', prev_model_version = 490)
# (model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False)
# all_price_df = model_pipeline(raceDB,config=wandb_config_static,sweep=False)

{'hidden_size': 256, 'stats': "['boxNumber', 'distance', 'dogAgeScaled', 'weightInKgScaled', 'hasEntryBoxNumberPlus1', 'hasEntryBoxNumberMinus1', 'rolling_box_win_percentage', 'dog_distance_min_1', 'dog_boxNumber_min_1', 'dog_runTimeNorm_min_1', 'dog_place_min_1', 'dog_resultMargin_min_1', 'dog_split_time_margin_min_1', 'dog_split_runTimeNorm_min_1', 'dog_time_1_min_1', 'dog_averageSpeed_min_1', 'dog_win_min_1', 'trainer_distance_min_365D', 'trainer_boxNumber_min_365D', 'trainer_runTimeNorm_min_365D', 'trainer_place_min_365D', 'trainer_resultMargin_min_365D', 'trainer_split_time_margin_min_365D', 'trainer_split_runTimeNorm_min_365D', 'trainer_time_1_min_365D', 'trainer_averageSpeed_min_365D', 'trainer_win_min_365D', 'dam_distance_min_365D', 'dam_boxNumber_min_365D', 'dam_runTimeNorm_min_365D', 'dam_place_min_365D', 'dam_resultMargin_min_365D', 'dam_split_time_margin_min_365D', 'dam_split_runTimeNorm_min_365D', 'dam_time_1_min_365D', 'dam_averageSpeed_min_365D', 'dam_win_min_365D', 'sir

100%|██████████| 2583/2583 [00:03<00:00, 728.00it/s] 


[(datetime.date(2022, 1, 3), datetime.date(2022, 12, 31))]
2022-12-31
Train examples [4934]


100%|██████████| 2582/2582 [00:01<00:00, 2003.77it/s]
1it [00:01,  1.29s/it]


Train examples [1773]
Train examples [1773]
Train examples [4934]
Train examples [4934]
here
input_size=tensor(17, device='cuda:0')


...

Loading model NZ-ancient-nexu-541, version 490
odict_keys(['h0', 'gru.weight_ih_l0', 'gru.weight_hh_l0', 'gru.bias_ih_l0', 'gru.bias_hh_l0', 'gru.weight_ih_l1', 'gru.weight_hh_l1', 'gru.bias_ih_l1', 'gru.bias_hh_l1', 'fc0.weight', 'fc0.bias', 'batch_norm.weight', 'batch_norm.bias', 'batch_norm.running_mean', 'batch_norm.running_var', 'batch_norm.num_batches_tracked', 'batch_norm_data.weight', 'batch_norm_data.bias', 'batch_norm_data.running_mean', 'batch_norm_data.running_var', 'batch_norm_data.num_batches_tracked', 'track_embedding.weight', 'extra_1.batch_norm.weight', 'extra_1.batch_norm.bias', 'extra_1.batch_norm.running_mean', 'extra_1.batch_norm.running_var', 'extra_1.batch_norm.num_batches_tracked', 'extra_1.fc0_p1.weight', 'extra_1.fc0_p1.bias', 'extra_1.fc0_p2.weight', 'extra_1.fc0_p2.bias', 'extra_1.fc0_p3.weight', 'extra_1.fc0_p3.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias', 'fc3.weight', 'fc3.bias'])


100%|██████████| 2583/2583 [00:00<00:00, 571920.35it/s]

filled =1022
empty  =1561
0.3956639566395664null_dog=0
GRUNetv3_extra_embedding(
  (gru): GRU(17, 256, num_layers=2, dropout=0.3)
  (relu): ReLU()
  (fc0): Linear(in_features=2611, out_features=2048, bias=True)
  (batch_norm): BatchNorm1d(17, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch_norm_data): BatchNorm1d(37, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (track_embedding): Embedding(1024, 50)
  (extra_1): GRUNetv3_simple_extra_data(
    (batch_norm): LazyBatchNorm1d(0, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU()
    (fc0_p1): LazyLinear(in_features=0, out_features=128, bias=True)
    (fc0_p1_drop): Dropout(p=0.3, inplace=False)
    (fc0_p2): Linear(in_features=128, out_features=64, bias=True)
    (fc0_p2_drop): Dropout(p=0.3, inplace=False)
    (fc0_p3): Linear(in_features=64, out_features=64, bias=True)
    (fc0_p3_drop): Dropout(p=0.3, inplace=False)
  )
  (relu0): ReLU()
  (drop1): Dropout(p=

ROI < 30,▁▁
ROI < 30 2,▁▁
accuracy,▁▁
accuracy2,▁
avg_win_price,▁▁
epoch,▁▁▁
favorite_accuracy,▁▁
flat_simple,▁▁
kelly roi,▁▁
kelly seq roi,▁▁
kelly seq roi < 30,▁▁


(GRUNetv3_extra_embedding(
   (gru): GRU(17, 256, num_layers=2, dropout=0.3)
   (relu): ReLU()
   (fc0): Linear(in_features=2611, out_features=2048, bias=True)
   (batch_norm): BatchNorm1d(17, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (batch_norm_data): BatchNorm1d(37, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
   (track_embedding): Embedding(1024, 50)
   (extra_1): GRUNetv3_simple_extra_data(
     (batch_norm): LazyBatchNorm1d(0, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (relu): ReLU()
     (fc0_p1): Linear(in_features=0, out_features=128, bias=True)
     (fc0_p1_drop): Dropout(p=0.3, inplace=False)
     (fc0_p2): Linear(in_features=128, out_features=64, bias=True)
     (fc0_p2_drop): Dropout(p=0.3, inplace=False)
     (fc0_p3): Linear(in_features=64, out_features=64, bias=True)
     (fc0_p3_drop): Dropout(p=0.3, inplace=False)
   )
   (relu0): ReLU()
   (drop1): Dropout(p=0.3, inplace=False)
   (fc1): Linear(in_

In [ ]:
def betfair_log_loss(output, classes, probabilites,reduce=True):
    label = torch.zeros_like(classes.data).scatter_(1, torch.argmax(classes.data, dim=1).unsqueeze(1), 1.)
    output_prob = F.softmax(output,dim=-1)
    log_loss = (-(label*output_prob.log()+(1-label)*(1-output_prob).log()))
    betfair_loss = (-(label*probabilites.log()+(1-label)*(1-probabilites).log()))
    return log_loss, betfair_loss, label

p = torch.rand(1,8)
y = torch.rand(1,8)
x = torch.rand(1,8)

betfair_log_loss(x,y,p)

(tensor([[0.0919, 0.1896, 0.2179, 2.0917, 0.1185, 0.1034, 0.0973, 0.1250]]),
 tensor([[0.0618, 0.6810, 1.2360, 1.2797, 0.1817, 0.7406, 1.0260, 1.7701]]),
 tensor([[0., 0., 0., 1., 0., 0., 0., 0.]]))

In [ ]:
profit_model = rnn_classes.GRUNetv3_profit(raceDB).to('cuda:0')

In [ ]:
raceDB.states

In [ ]:
new_batches = []
for batch in batch_races:
    batch_dict = {state:[] for state in raceDB.states}
    for race in batch:
        batch_dict[race.state].append(race)
    new_batch = list(batch_dict.values())
    new_batches.append(new_batch)
    for k,v in batch_dict.items():
        print(f"{k} races {len(v)}")
raceDB.batches['new_batch_races'] = new_batches

In [ ]:
[print(len(batch)) for batch in raceDB.batches['new_batch_races']]

In [ ]:
list(batch_dict.values())

In [ ]:
importlib.reload(rnn_tools.rnn_classes)
importlib.reload(rnn_classes)
importlib.reload(training_testing_gru_extra_data)
(model,dataset, optimizer) = model_pipeline(raceDB,config=wandb_config_static,sweep=False)

# profit_tensor =  model_pipeline(raceDB,config=wandb_config_static,sweep=False)

In [ ]:
def print_grad_fn(tensor):
    print(tensor)
    if tensor is not None:
        if tensor is not None:
            for t in tensor.next_functions:
                if t[0] is not None:
                    # print(f"{t[0]=}")
                    print_grad_fn(t[0])

# Usage:
# Assuming `output` is your tensor
# print_grad_fn(output)

In [ ]:
profit_tensor.grad_fn.next_functions

In [ ]:
profit_tensor.shape

In [ ]:
print_grad_fn(profit_tensor[0].grad_fn)

In [ ]:
profit_tensor.grad_fn.next_functions

In [ ]:
len_test = len(raceDB.test_race_ids)
test_idx = range(0,len_test)
race = raceDB.get_test_input(test_idx)

In [ ]:
for r in race:
    print(r.prices)
    print(r.raceid)
    _, actual = torch.max(r.classes, 0)
    onehot_win = F.one_hot(actual, num_classes=8)
    print(onehot_win)
    print(r.win_price_weight)
    print(r.raw_margins)
    print(r.race_date)
    print(r.track_name)
    print('----------------------------')

In [ ]:
import plotly.graph_objs as go
import plotly.subplots as sp

hidden_outs = []
margins = []
for race in raceDB.dogsDict['410839665'].races.values():
    hidden_outs.append(race.hidden_out.detach())
    margins.append(race.margin)
# Assuming hidden_states is your list of tensors
hidden_states = [x[0:256] for x in hidden_outs]
hidden_states = [hidden_state.reshape(16, 16) for hidden_state in hidden_states]
# Create a heatmap for each step
frames = [go.Frame(
    data=[go.Heatmap(z=hidden_state.tolist(), colorscale='Viridis')],
    name=str(i)
) for i, hidden_state in enumerate(hidden_states)]

# Create a slider
sliders = [dict(
    steps=[dict(method='animate',
                args=[[frame['name']]],
                label=str(i)) for i, frame in enumerate(frames)],
    transition=dict(duration=300, easing='cubic-in-out'),
    active=0,
)]

# Create a layout
layout = go.Layout(
    sliders=sliders,
    updatemenus=[dict(type='buttons',
                      showactive=False,
                      buttons=[dict(label='Play',
                                    method='animate',
                                    args=[None, {"frame": {"duration": 500, "redraw": False},
                                                 "fromcurrent": True,
                                                 "transition": {"duration": 300,
                                                                "easing": "quadratic-in-out"}}]),
                                  dict(label='Stop',
                                       method='animate',
                                       args=[[None], {"frame": {"duration": 0, "redraw": False},
                                                      "mode": "immediate",
                                                      "transition": {"duration": 0}}])])],
)

# Create a figure
fig = go.Figure(data=frames[0]['data'], layout=layout, frames=frames)

# Display the figure
fig.show()

In [ ]:
import plotly.graph_objs as go
import plotly.subplots as sp
from plotly.subplots import make_subplots
hidden_inits= [hidden_state[1,:].reshape(16, 16) for hidden_state in raceDB.hidden_state_inits]

dogs = ['366590730','410839665']

hiddens = []
margins = []
for dog in dogs:
    hidden_outs = []
    margin = []
    for race in raceDB.dogsDict[dog].races.values():

        hidden_outs.append(race.hidden_out.detach())
        margin.append(race.margin)
    hiddens.append(hidden_outs)
    margins.append(margin)
    
min_len = min([len(x) for x in hiddens])
print(f"{min_len=}")
hidden_states = [[hidden_state[0:256].cpu().numpy().reshape(16, 16) for hidden_state in dog[0:min_len]] for dog in hiddens]
print(f"{[len(x) for x in hidden_states]=}")
# Create a heatmap for each step
global_min = min(hidden_state.min() for hidden_state in hidden_states[0])
global_max = max(hidden_state.max() for hidden_state in hidden_states[0])

print(global_min, global_max)
# print(hidden_states[0])
# Create a heatmap for each step with constant scale
# Create a subplot for each dog
# Create a subplot for each dog with an additional row for the line plot
fig = make_subplots(rows=4, cols=1)

# Create a heatmap and line plot for each dog
for i, dog_hidden_states in enumerate(hidden_states):
    # Create heatmap
    heatmap = go.Heatmap(z=dog_hidden_states[0].tolist(), zmin=global_min, zmax=global_max, colorscale='Viridis')
    fig.add_trace(heatmap, row=2*i+2, col=1)

    # Create line plot
    line_plot = go.Scatter(y=[sum(abs(state.flatten())) for state in dog_hidden_states], mode='lines',showlegend=False)
    fig.add_trace(line_plot, row=2*i+1, col=1)

frames = []

# Iterate over dogs and their corresponding hidden states
for j in range(min_len):
    frame_data = []
    for i, dog_hidden_states in enumerate(hidden_states):
        hidden_state = dog_hidden_states[j]
        heatmap = go.Heatmap(z=hidden_state.tolist(), zmin=global_min, zmax=global_max, colorscale='Viridis')
        line_plot = go.Scatter(y=[sum(abs(state.flatten())) for state in dog_hidden_states[:j+1]], mode='lines',showlegend=False)
        frame_data.extend([heatmap, line_plot])
    frame = go.Frame(data=frame_data, name=str(j))  # Create a frame for each time step
    frames.append(frame)

# Define the slider
# Define the slider
sliders = [dict(steps=[dict(method='animate',
                            args=[[f.name], {"mode": "immediate",
                                             "frame": {"duration": 100, "redraw": True},
                                             "transition": {"duration": 0}}],
                            label=f.name) for f in frames],
                active=0)]

# Update layout
fig.update_layout(
    updatemenus=[dict(type='buttons',
                      showactive=False,
                      buttons=[dict(label='Play',
                                    method='animate',
                                    args=[None, {"frame": {"duration": 100, "redraw": False},
                                                 "fromcurrent": True,
                                                 "mode": "immediate",
                                                 "transition": {"duration": 0,
                                                                "easing": "quadratic-in-out"}}]),
                                  dict(label='Stop',
                                       method='animate',
                                       args=[[None], {"frame": {"duration": 0, "redraw": False},
                                                      "mode": "immediate",
                                                      "transition": {"duration": 0}}])])],
    sliders=sliders
)
# Update frames
fig.frames = frames

# Display the figure
fig.show()
import plotly.io as pio

# Save the figure as an interactive html file
pio.write_html(fig, 'hidden_simple_double.html')

In [ ]:
# Create a line plot for each dog
fig = go.Figure()

for i, dog_hidden_states in enumerate(hidden_states):
    # Calculate the absolute sum of hidden states for each time step
    abs_sum = [sum(abs(state.flatten())) for state in dog_hidden_states]
    print(abs_sum)

    # Create line plot with x values specified
    line_plot = go.Scatter(x=list(range(len(dog_hidden_states))), y=abs_sum, mode='lines', name=f'Dog {i+1}')
    fig.add_trace(line_plot)

# Update layout
fig.update_layout(
    title="Absolute Sum of Hidden States Over Time",
    xaxis_title="Time Step",
    yaxis_title="Absolute Sum of Hidden States",
    legend_title="Dogs",
)

# Display the figure
fig.show()

In [ ]:
from bokeh.layouts import column
from bokeh.models import Slider, CustomJS
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.io import curdoc
import numpy as np

# Assuming hidden_states is a list of 2D numpy arrays
hidden_states = [[hidden_state[0:256].cpu().numpy().reshape(16, 16) for hidden_state in dog[0:min_len]] for dog in hiddens]

# Create a ColumnDataSource to hold the data for the current frame
source = ColumnDataSource(data={'image': [hidden_states[0][0]]})

# Create a figure and add an image renderer
p = figure(x_range=(0, 1), y_range=(0, 1), width=400, height=400, match_aspect=True)
p.image(image='image', x=0, y=0, dw=1, dh=1, source=source)

# Create a slider that goes from the first to the last frame
slider = Slider(start=0, end=len(hidden_states[0])-1, value=0, step=1)

# Define a callback function to update the ColumnDataSource's data when the slider is changed
callback = CustomJS(args=dict(source=source, states=hidden_states[0]), code="""
    var i = cb_obj.value;
    source.data = {'image': [states[i]]};
""")

# Attach the callback to the slider
slider.js_on_change('value', callback)

# Display the figure and slider
layout = column(p, slider)
show(layout)

In [ ]:
for i in raceDB.hidden_state_inits:
    print(i[0,0])

In [ ]:
hidden_state_init = model.h0
raceDB.reset_hidden_w_param(hidden_state_init,num_layers=2, hidden_size=256)

In [ ]:
raceDB.dogsDict['408750377'].hidden